In [1]:
!pip install tensorflow==2.5.0

In [2]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
data = []
with open("scraped_all.json", "r") as f:
    for line in f:
        data.append(json.loads(line))
# print(data[0].get("lines"))
corpus=[]
for i in range(len(data)):
    poem = data[i]
    for j in range(len(poem.get("lines"))):
        corpus.append(poem.get("lines")[j])
# corpus= corpus.remove('')
for k in range(len(corpus)):
    corpus[k] = corpus[k].replace('\n',"")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# print(tokenizer.word_index)
print(total_words)
# print(corpus)


7641


In [5]:
import numpy as np

In [6]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(tokenizer.word_index)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=20, verbose=1)
#print model.summary()
print(model)

{'है': 1, 'में': 2, 'की': 3, 'से': 4, 'के': 5, 'का': 6, 'को': 7, 'भी': 8, 'तो': 9, 'हैं': 10, 'मैं': 11, 'हो': 12, 'नहीं': 13, 'पर': 14, 'ही': 15, 'एक': 16, 'और': 17, 'कर': 18, 'जो': 19, 'न': 20, 'ये': 21, 'है।': 22, 'वो': 23, 'ना': 24, 'अब': 25, 'कुछ': 26, 'कोई': 27, 'था': 28, 'हम': 29, 'मेरे': 30, 'हर': 31, 'तुम': 32, 'दिल': 33, '।': 34, 'कभी': 35, 'सब': 36, 'फिर': 37, 'मन': 38, 'क्या': 39, 'हूं': 40, 'तू': 41, 'जीवन': 42, 'ने': 43, 'रही': 44, 'मुझे': 45, 'रहा': 46, 'घर': 47, 'इस': 48, 'अपने': 49, 'क्यों': 50, 'थी': 51, 'रहे': 52, 'यह': 53, 'आज': 54, 'हूँ': 55, 'जब': 56, 'हैं।': 57, 'तेरे': 58, 'साथ': 59, 'माँ': 60, 'मेरा': 61, 'मेरी': 62, 'अपना': 63, 'बात': 64, 'कि': 65, 'प्यार': 66, 'आया': 67, 'लिए': 68, 'तक': 69, 'गया': 70, 'जाने': 71, 'थे': 72, 'बन': 73, 'सी': 74, 'हुआ': 75, 'बहुत': 76, 'ले': 77, 'या': 78, 'दो': 79, 'खुद': 80, 'अपनी': 81, 'दे': 82, 'दर्द': 83, 'किसी': 84, 'पे': 85, 'रह': 86, 'उस': 87, 'दिन': 88, 'जिंदगी': 89, 'तेरी': 90, 'करते': 91, 'जाता': 92, 'गए': 93, 'रात': 9

Epoch 1/20
966/966 [==============================] - 69s 69ms/step - loss: 7.4183 - accuracy: 0.0428
Epoch 2/20
966/966 [==============================] - 73s 75ms/step - loss: 6.3479 - accuracy: 0.0971
Epoch 3/20
966/966 [==============================] - 72s 74ms/step - loss: 5.1780 - accuracy: 0.1688
Epoch 4/20
966/966 [==============================] - 69s 71ms/step - loss: 3.9972 - accuracy: 0.2638
Epoch 5/20
966/966 [==============================] - 68s 70ms/step - loss: 3.1175 - accuracy: 0.3682
Epoch 6/20
966/966 [==============================] - 66s 68ms/step - loss: 2.5238 - accuracy: 0.4635
Epoch 7/20
966/966 [==============================] - 69s 72ms/step - loss: 2.1412 - accuracy: 0.5262
Epoch 8/20
966/966 [==============================] - 63s 65ms/step - loss: 1.8869 - accuracy: 0.5723
Epoch 9/20
966/966 [==============================] - 63s 65ms/step - loss: 1.7534 - accuracy: 0.5945
Epoch 10/20
966/966 [==============================] - 72s 75ms/step - loss: 1.635

In [7]:
seed_text = "स्कूल के दिन"
next_words = 100

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

/Users/harsitsinha/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


स्कूल के दिन में जाने की भी डरती है नहीं मिला था वो था वहाँ जाना छोड दूँ आओ। है । ॥ थे। अब क्या फ़ैसला अब कोई नही में कहना है। हो आओ आओ पाता हो जैसे छोड़ कोई फना ।। मनु के वास्ते आओ आओ आया था लिया जाता है ख़्वाब । ॥ ।। ।। है। । ॥ है। दिल का एक बड़ा धक्का लगा हमने सकता है। के कारण सुंदर शिक्षा ज्ञान व का हैं बटेर।। है। में तुम खूब मजा पूरी दिया था। देख सकता था। आया है तुम है मुझसे सकता था। था सकता गलियों है जो मेरा


In [8]:
random(1,5)

NameError: name 'random' is not defined